# SW중심대학 디지털 경진대회_SW와 생성AI의 만남 : AI부문
 - 이 AI 경진대회에서는 5초 분량의 오디오 샘플에서 진짜 사람 목소리와 AI가 생성한 가짜 목소리를 정확하게 구분할 수 있는 모델을 개발하는 것이 목표입니다.
 - 이 작업은 보안, 사기 감지 및 오디오 처리 기술 향상 등 다양한 분야에서 매우 중요합니다.

## Imports
모델 학습 및 추론에 사용할 라이브러리들을 불러옵니다.

In [ ]:
import os
import copy
import random

import torch
import torchaudio

import numpy as np
import pandas as pd

from torch import nn
import torch.nn.functional as F
import torchaudio.transforms as T
import torchaudio.pipelines as pipelines
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

### Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0

device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_NUM)
    device = torch.device("cuda")
print("INFO: Using device -", device)

## Config
- 딥러닝 모델을 학습하기 전에 설정해야하는 다양한 매개변수를 정의하는 설정 클래스입니다.
- 클래스를 사용하여 학습에 필요한 설정 값을 미리 지정합니다.

##### 오디오 신호
- 우리가 듣는 소리는 공기의 압력 변화로, 이것을 디지털 신호로 변환한 것이 오디오 신호입니다.
- 이 신호는 시간에 따라 변하는 진폭 값을 가지고 있습니다.

In [ ]:
class Config:
    """ Configuration Class """
    SEED = 20240719  # 재현성을 위해 랜덤 시드 고정
    NB_NAME = "wav2vec_pre_transfer_learning"  # ipython 노트북 이름 지정
    
    """ SR(Sample Rate)
    - 오디오 데이터의 샘플링 레이트를 설정합니다.
    - 높은 샘플링 레이트는 더 높은 주파수의 소리를 캡처할 수 있지만, 처리에 더 많은 계산 자원이 필요합니다.
    - 오디오 데이터의 초당 샘플 수를 정의합니다.
    """
    SR = 16000

    """ ROOT_FOLDER
    - 데이터셋의 루트 폴더 경로를 설정합니다.
    """
    ROOT_FOLDER = os.path.join(".", "data")
    
    """ BATCH_SIZE
    - 학습 시 한 번에 처리할 데이터 샘플의 수를 정의합니다
    - 큰 배치 크기는 메모리 사용량을 증가시키지만, 학습 속도를 높입니다.
    """
    BATCH_SIZE = 128
    
    """ LR (Learning Rate)
    - 모델의 가중치를 업데이트할 때 사용되는 학습 속도를 정의합니다.
    - 학습률이 너무 크면 학습이 불안정해질 수 있고, 너무 작으면 학습 속도가 느려집니다.
    """
    LR = 1e-5

In [ ]:
def seed_everything(seed):
    """ Fixed RandomSeed
    아래의 코드는 머신러닝이나 딥러닝 모델을 훈련할 때, 결과의 재현성을 보장하기 위해 사용되는 함수입니다.
    이 함수는 다양한 랜덤 시드를 고정하여, 실행할 때마다 동일한 결과를 얻기 위해 사용됩니다.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(Config.SEED)  # Seed 고정

## Dataset

In [ ]:
from torchvision.datasets import utils
from sklearn.model_selection import train_test_split as split

utils.tqdm = tqdm


class VoiceDataset(Dataset):
    download_url = "https://drive.usercontent.google.com/download?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&export=download&authuser=0&confirm=t&uuid=c40c278b-d74b-4b75-bc79-09e8a3ccffa4&at=APZUnTUvIVFVM9gjGNUCmDb4YZCy%3A1719807236671"

    @classmethod
    def download(cls, root='./data', filename="download.zip", md5=None):
        cls.download_root = root
        filepath = os.path.join(root, filename)
        if not os.path.exists(filepath):
            utils.download_and_extract_archive(cls.download_url, root, root, filename, md5)
            print("Extraction completed.")
        else:
            print(f"File already exists in {filepath}")

    @property
    def get_dataset_path(self):
        filename = "train.csv" if self.is_train else "test.csv"
        if self.custom_csv:
            filename = self.custom_csv + ".csv"
        return os.path.join(self.download_root, filename)

    @property
    def submission_form_path(cls):
        return os.path.join(cls.download_root, "sample_submission.csv")

    def __init__(self, root="./data", train=True, split_ratio=1, transform=None, custom_csv=None):
        """
        Voice Dataset for Contrastive Learning
        
        :param root: The path to the data directory
        :param train: is train or test
        :param split_ratio: split ratio for train(can be 0.5 or above) and valid(can be lower than 0.5) set
        :param transform: data transformer
        :param target_transform: label transformer
        """
        super().__init__()
        self.download(root)
        self.download_root = root
        self.is_train = train
        self.custom_csv = custom_csv
        self.dataset_name = ("train" if train else "test") if not custom_csv else custom_csv

        raw_data = self._load_data(self.get_dataset_path, split_ratio if split_ratio >= 0.5 else 1-split_ratio)
        if not self.is_train or split_ratio >= 0.5:
            self.raw_data, _ = raw_data
        else:
            _, self.raw_data = raw_data
            if "train" not in self.dataset_name:
                print(f"Warning: The name of dataset should start with 'train' for training set. (current - {self.dataset_name})")
            self.dataset_name = self.dataset_name.replace("train", "valid")

        self.data = self.raw_data['path'].tolist()

        if 'label' in self.raw_data.columns:
            self.label = [(0, 1) if lb == 'real' else (1, 0) for lb in self.raw_data['label'].tolist()]
        else:
            if 'real' in self.raw_data.columns and 'fake' in self.raw_data.columns:
                f_label = self.raw_data['fake'].tolist()
                r_label = self.raw_data['real'].tolist()
                self.label = list(zip(f_label, r_label))
            else:
                self.label = None

        self.transforms(transform)

    @staticmethod
    def _load_data(dataset_path, split_ratio=1):
        random_state = 1  # fixed random_state

        df = pd.read_csv(dataset_path)

        if split_ratio == 1 or split_ratio == 0:
            return (df, None) if split_ratio == 1 else (None, df)

        if 'label' in df.columns:
            df1, df2, _, _ = split(df, df['label'], test_size=1-split_ratio, random_state=random_state)
        else:
            df1, df2 = split(df, test_size=1-split_ratio, random_state=random_state)
        return df1, df2

    def transforms(self, transform=None):
        if transform is not None:
            if not isinstance(transform, list) and not isinstance(transform, tuple):
                transform = [transform]
            for t in transform:
                self.data, self.label = t(self.data, self.label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.label is not None:
            return self.data[index], self.label[index]
        return self.data[index]

In [ ]:
split_ratio = 0.8

# Train Set
train_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=split_ratio)
train_noise_type1 = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=split_ratio, custom_csv="train_noise_type1")
train_augmented = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=split_ratio, custom_csv="train_augmented")

# Valid Set
valid_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=1-split_ratio)
valid_noise_type1 = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=1-split_ratio, custom_csv="train_noise_type1")
valid_augmented = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=1-split_ratio, custom_csv="train_augmented")

# Unlabeled Set
unlabeled_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=False, custom_csv="unlabeled_data")

# Test Set
test_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=False)

print(f"Loaded Dataset - train({len(train_dataset)}), valid({len(valid_dataset)}), unlabeled({len(unlabeled_dataset)}) test({len(test_dataset)})")
print("Query Dataset for checking:", train_dataset[0])
train_dataset.raw_data

#### Data Transformation

In [ ]:
audio_cache_dir = "audio_cache"  # audio cache directory

if not os.path.isdir(audio_cache_dir):
    os.mkdir(audio_cache_dir)
if not os.path.isdir(os.path.join(audio_cache_dir, Config.NB_NAME)):
    os.mkdir(os.path.join(audio_cache_dir, Config.NB_NAME))

In [ ]:
def to_waveform(dataset_name):
    convert_path = lambda path: os.path.join(Config.ROOT_FOLDER, *path.replace("./", "").split("/"))
    embedding_path = os.path.join(audio_cache_dir, Config.NB_NAME.replace("_pre", ""), dataset_name+"_embedding.pt")

    def convert(datas, labels):
        if not os.path.isfile(embedding_path):
            new_datas = []
    
            for path in tqdm(datas):
                waveform, sample_rate = torchaudio.load(convert_path(path), normalize=True)
                if sample_rate != Config.SR:
                    resampler = torchaudio.transforms.Resample(sample_rate, Config.SR)
                    waveform = resampler(waveform)
                new_datas.append(waveform)

            torch.save(new_datas, embedding_path)
            print("INFO: Voice Embedding saved.")
        else:
            new_datas = torch.load(embedding_path)
            print(f"INFO: Pretrained Voice Embedding loaded. - id({id(new_datas)}), type({type(new_datas)})")

        return new_datas, labels

    return convert

In [ ]:
def pad_tensor(max_len=None):
    def pad(datas, labels):
        padding_size = max_len
        if padding_size is None:
            padding_size = max(t.size(1) for t in datas)
        return [F.pad(t, (0, padding_size - t.size(1))) for t in datas], labels
    return pad

In [ ]:
to_tensor = lambda d, l: (d, list(map(torch.tensor, l)))  # label to tensor

for dataset in [train_dataset, train_noise_type1, train_augmented]:
    dataset.transforms(transform=[to_waveform(dataset.dataset_name), to_tensor])

for dataset in [valid_dataset, valid_noise_type1, valid_augmented]:
    dataset.transforms(transform=[to_waveform(dataset.dataset_name), to_tensor])

unlabeled_dataset.transforms(transform=to_waveform(unlabeled_dataset.dataset_name))
test_dataset.transforms(transform=to_waveform(test_dataset.dataset_name))

In [ ]:
for (data, label), i in zip(train_dataset, range(5)):
    print(f"Dataset {i}: {label}", data)

In [ ]:
for (data, label), i in zip(valid_dataset, range(5)):
    print(f"Dataset {i}: {label}", data)

In [ ]:
for dataset, i in zip(test_dataset, range(5)):
    print(f"Dataset {i}:", dataset)

## Model Definition & Training

### 0. Wav2Vec2 Embedding

[TorchAudio Models](https://pytorch.org/audio/stable/models.html)

[TorchAudio Pretrained Models](https://pytorch.org/audio/stable/pipelines.html#module-torchaudio.pipelines)

In [ ]:
wav2vec_bundle = pipelines.WAV2VEC2_ASR_BASE_960H
wav2vec = wav2vec_bundle.get_model()
print(f"Wav2Vec2 Model Loaded. Bundle: {wav2vec_bundle}")  # Wav2Vec2 Model uses sample rate 16kHz
wav2vec.to(device)

In [ ]:
def to_embedding(dataset_name):
    embedding_path = os.path.join(audio_cache_dir, Config.NB_NAME, dataset_name+"_embedding.pt")

    def convert(datas, labels):
        if not os.path.isfile(embedding_path):
            new_datas = []

            for waveform in tqdm(datas):
                embedding, _ = wav2vec(waveform)
                new_datas.append(embedding)

            torch.save(new_datas, embedding_path)
            print("INFO: Voice Embedding saved.")
        else:
            new_datas = torch.load(embedding_path)
            print(f"INFO: Pretrained Voice Embedding loaded. - id({id(new_datas)}), type({type(new_datas)})")

        return new_datas, labels

    return convert

In [ ]:
for dataset in [train_dataset, train_noise_type1, train_augmented]:
    dataset.transforms(transform=to_embedding(dataset.dataset_name))

for dataset in [valid_dataset, valid_noise_type1, valid_augmented]:
    dataset.transforms(transform=to_embedding(dataset.dataset_name))

unlabeled_dataset.transforms(transform=to_embedding(unlabeled_dataset.dataset_name))
test_dataset.transforms(transform=to_embedding(test_dataset.dataset_name))

### 1. Encoder-Decoder Feature Extractor

In [ ]:
class FeatureExtractor(nn.Module):
    """ Feature Extractor Model """

    def __init__(self, embedding_size, latent_size):
        super().__init__()
        self.encoder = nn.Linear(embedding_size, latent_size)
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, embedding_size//2),
            nn.Linear(embedding_size//2, embedding_size)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        latent = self.relu(self.encoder(x))
        restored = self.relu(self.decoder(latent))
        return latent, restored


### 2. Real-Fake Voice Discriminator

In [ ]:
class RFVoiceDiscriminator(nn.Module):
    """ Real-Fake Voice Discriminator Model """

    def __init__(self, embedding_size, latent_size):
        super().__init__()
        self.latent_size = latent_size
        
        self.feature = FeatureExtractor(embedding_size=embedding_size, latent_size=latent_size)
        self.fc1 = nn.Linear(latent_size, 1)
        self.fc2 = nn.Linear(embedding_size, 1)

    def forward(self, x):
        latent, restored = self.feature(x)
        return restored, self.fc1(latent), self.fc2(restored)

In [ ]:
rf_disc = RFVoiceDiscriminator(embedding_size=512, latent_size=32)
rf_disc.to(device)

In [ ]:
class RFVoiceDiscriminatorConfig:
    num_epochs = 50
    log_interval = 5
    
    # BinaryCrossEntropy
    criterion_b = nn.BCELoss().to(device)
    
    # Mean Squared Error
    criterion_m = nn.MSELoss().to(device)
    
    # Adam optimizer
    optimizer = torch.optim.Adam(params=rf_disc.parameters(), lr=Config.LR)
    
    # Dataset
    train_dataset = copy.deepcopy(train_dataset)
    train_dataset.data.extend(train_noise_type1.data)
    train_dataset.label.extend(train_noise_type1.label)
    
    valid_dataset = copy.deepcopy(valid_dataset)
    valid_dataset.data.extend(valid_noise_type1.data)
    valid_dataset.label.extend(valid_noise_type1.label)
    
    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

In [ ]:
rf_disc_config = RFVoiceDiscriminatorConfig

num_epochs = rf_disc_config.num_epochs
log_interval = rf_disc_config.log_interval
train_len, valid_len = map(len, (rf_disc_config.train_loader, rf_disc_config.valid_loader))

In [ ]:
epochs = tqdm(range(1, num_epochs+1), desc="Running Epochs")
with tqdm(total=train_len, desc="Training") as train_progress, tqdm(total=valid_len, desc="Validation") as valid_progress:
    optimizer = rf_disc_config.optimizer
    criterion_b = rf_disc_config.criterion_b
    criterion_m = rf_disc_config.criterion_m
    
    for epoch in epochs:
        train_progress.reset(total=train_len)
        valid_progress.reset(total=valid_len)

        # Train
        rf_disc.train()
        for i, (features, labels) in enumerate(rf_disc_config.train_loader):
            optimizer.zero_grad()

            features = features.to(device)
            restoreds, preds1, preds2 = rf_disc(features)
            labels = torch.argmax(labels, dim=1).float().to(device)

            loss = criterion_m(features, restoreds) + criterion_b(preds1, labels) + criterion_b(preds2, labels)
            loss.backward()
            optimizer.step()

            train_progress.update(1)
            print(f"\rEpoch [{epoch}/{num_epochs}], Step [{i+1}/{train_len}], Loss: {loss.item():.6f}", end="")

        val_acc, val_loss = 0, 0

        # Validation
        rf_disc.eval()
        with torch.no_grad():
            for (features, labels) in rf_disc_config.valid_loader:
                _, preds1, preds2 = rf_disc(features)
                labels = torch.argmax(labels, dim=1).float().to(device)

                val_loss += (criterion_b(preds1, labels).item() + criterion_b(preds2, labels).item()) / (2 * valid_len)
                val_acc += (((preds1 >= 0.5).float() == labels).sum() + ((preds2 >= 0.5).float() == labels).sum()) / (2 * valid_len)

                valid_progress.update(1)

        print(f"\rEpoch [{epoch}/{num_epochs}], Step [{train_len}/{train_len}], Loss: {loss.item():.6f}, "
            + f"Valid Acc: {val_acc:.6%}, Valid Loss: {val_loss:.6f}", end="\n" if epoch % log_interval == 0 or epoch == num_epochs else "")

### 3. Voice Presence Detector

In [ ]:
class VoiceDetector(nn.Module):
    """ Voice presence detection module """

    def __init__(self, embedding_size, latent_size):
        super().__init__()
        self.latent_size = latent_size
        
        self.feature = FeatureExtractor(embedding_size=embedding_size, latent_size=latent_size)
        self.fc1 = nn.Linear(latent_size, 1)
        self.fc2 = nn.Linear(embedding_size, 1)

    def forward(self, x):
        latent, restored = self.feature(x)
        return restored, self.fc1(latent), self.fc2(restored)

In [ ]:
detector = VoiceDetector(embedding_size=512, latent_size=32)
detector.to(device)

In [ ]:
class VoiceDetectorConfig:
    num_epochs = 5
    log_interval = 1

    # BinaryCrossEntropy
    criterion_b = nn.BCELoss().to(device)

    # Mean Squared Error
    criterion_m = nn.MSELoss().to(device)

    # Adam optimizer
    optimizer = torch.optim.Adam(params=rf_disc.parameters(), lr=Config.LR)

    # Dataset
    train_dataset = unlabeled_dataset
    valid_dataset = valid_augmented
    
    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
    valid_loader = DataLoader(valid_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

In [ ]:
vdet_config = VoiceDetectorConfig

num_epochs = vdet_config.num_epochs
log_interval = vdet_config.log_interval
train_len, valid_len = map(len, (vdet_config.train_loader, vdet_config.valid_loader))

In [ ]:
epochs = tqdm(range(1, num_epochs+1), desc="Running Epochs")
with tqdm(total=train_len, desc="Training") as train_progress, tqdm(total=valid_len, desc="Validation") as valid_progress:
    optimizer = vdet_config.optimizer
    criterion_b = vdet_config.criterion_b
    criterion_m = vdet_config.criterion_m

    for epoch in epochs:
        train_progress.reset(total=train_len)
        valid_progress.reset(total=valid_len)

        # Train
        detector.train()
        for i, features in enumerate(vdet_config.train_loader):
            optimizer.zero_grad()

            features = features.to(device)
            restoreds, preds1, preds2 = detector(features)
            labels = torch.softmax(features, dim=3)
            labels = (torch.max(labels, dim=1)[0] >= 0.5).float

            loss = criterion_m(features, restoreds) + criterion_b(preds1, labels) + criterion_b(preds2, labels)
            loss.backward()
            optimizer.step()

            train_progress.update(1)
            print(f"\rEpoch [{epoch}/{num_epochs}], Step [{i+1}/{train_len}], Loss: {loss.item():.6f}", end="")

        val_acc, val_loss = 0, 0

        # Validation
        detector.eval()
        with torch.no_grad():
            for (features, labels) in vdet_config.valid_loader:
                _, preds1, preds2 = detector(features)
                labels = (labels.to(device).sum(dim=1) > 0).float()

                val_loss += (criterion_b(preds1, labels).item() + criterion_b(preds2, labels).item()) / (2 * valid_len)
                val_acc += (((preds1 >= 0.5).float() == labels).sum() + ((preds2 >= 0.5).float() == labels).sum()) / (2 * valid_len)

                valid_progress.update(1)

        print(f"\rEpoch [{epoch}/{num_epochs}], Step [{train_len}/{train_len}], Loss: {loss.item():.6f}, "
            + f"Valid Acc: {val_acc:.6%}, Valid Loss: {val_loss:.6f}", end="\n" if epoch % log_interval == 0 or epoch == num_epochs else "")

### 4. Total FakeVoiceDetectionModel

In [ ]:
from sklearn.metrics import roc_auc_score

def multi_label_auc(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [ ]:
class FakeVoiceDetectionModel(nn.Module):
    """ Fake Voice Detection Model """

    def __init__(self, hidden_size):
        super().__init__()
        self.latent_size = rf_disc.latent_size + detector.latent_size
        self.hidden_size = hidden_size
        
        self.encoder1 = rf_disc.feature.encoder
        self.encoder2 = detector.feature.encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.latent_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, 2)
        )

    def forward(self, x):
        latent1, latent2 = self.encoder1(x), self.encoder2(x)
        latent = latent1.cat(latent2, dim=1)
        return self.classifier(latent)

In [ ]:
model = FakeVoiceDetectionModel(hidden_size=256)
model.to(device)

In [ ]:
class FakeVoiceDetectionModelConfig:
    num_epochs = 5
    log_interval = 1

    # BinaryCrossEntropy
    criterion = nn.BCELoss().to(device)

    # Adam optimizer
    optimizer = torch.optim.Adam(params=rf_disc.parameters(), lr=Config.LR)

    # Dataset
    train_dataset = copy.deepcopy(train_dataset)
    train_dataset.data.extend(train_noise_type1.data)
    train_dataset.data.extend(train_augmented.data)
    self_train_dataset = unlabeled_dataset
    valid_dataset = copy.deepcopy(valid_dataset)
    valid_dataset.data.extend(valid_noise_type1.data)
    valid_dataset.data.extend(valid_augmented.data)

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    self_train_loader = DataLoader(self_train_dataset, batch_size=1, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

model_config = FakeVoiceDetectionModelConfig

In [ ]:
last_val_score = 0
self_labels = []  # do not shuffle data
num_epochs = model_config.num_epochs
log_interval = model_config.log_interval
train_len, self_len, valid_len = map(len, (model_config.train_loader, model_config.self_train_loader, model_config.valid_loader))

epochs = tqdm(range(1, num_epochs+1), desc="Total Model Running Epochs")
with (
    tqdm(total=train_len, desc="Self-Supervised Learning") as self_progress,
    tqdm(total=train_len, desc="Supervised Learning") as train_progress,
    tqdm(total=valid_len, desc="Validation") as valid_progress
    ):
    for epoch in epochs:
        train_progress.reset(total=train_len)
        self_progress.reset(total=self_len)
        valid_progress.reset(total=valid_len)

        # Self-Supervised Learning
        model.train()
        for i, inputs in enumerate(zip(model_config.self_train_loader, self_labels)):
            optimizer.zero_grad()

            features, labels = (data.float().to(device) for data in inputs)
            outputs = model(features)

            loss = model_config.criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            self_progress.update(1)
            print(f"\rEpoch [{epoch}/{num_epochs}], Step [{i+1}/{train_len}], Loss: {loss.item():.6f}", end="")

        # Supervised Learning
        model.train()
        for i, inputs in enumerate(model_config.train_loader):
            optimizer.zero_grad()

            features, labels = (data.float().to(device) for data in inputs)
            outputs = model(features)

            loss = model_config.criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_progress.update(1)
            print(f"\rEpoch [{epoch}/{num_epochs}], Step [{i+1}/{train_len}], Loss: {loss.item():.6f}", end="")

        # Self-Supervised Labeling
        self_loss, _self_labels, self_outputs = 0, [], []
        model.eval()
        with torch.no_grad():
            for inputs in zip(model_config.self_train_loader, self_labels):
                features, labels = (data.float().to(device) for data in inputs)
                predicted = model(features)

                self_loss += model_config.criterion(predicted, labels).item() / self_len
                _self_labels.append(labels.cpu().numpy())
                self_outputs.append(predicted.cpu().numpy())

                self_progress.update(1)

        self_score = multi_label_auc(np.concatenate(_self_labels, axis=0), np.concatenate(self_outputs, axis=0))
        self_labels = (_self_labels >= 0.5).float()

        print(f"\rEpoch [{epoch}/{num_epochs}], Loss: {loss.item():.6f}, Self Acc: {self_score:.6%}, Self Loss: {self_loss:.6f}", end="")

        # Validation
        model.eval()
        val_loss, val_labels, val_outputs = 0, [], []
        with torch.no_grad():
            for inputs in model_config.valid_loader:
                features, labels = (data.float().to(device) for data in inputs)
                predicted = model(features)

                val_loss += model_config.criterion(predicted, labels).item() / valid_len
                val_labels.append(labels.cpu().numpy())
                val_outputs.append(predicted.cpu().numpy())

                valid_progress.update(1)

        last_val_score = multi_label_auc(np.concatenate(val_labels, axis=0), np.concatenate(val_outputs, axis=0))  # Calculate AUC score

        print(f"\rEpoch [{epoch}/{num_epochs}], Loss: {loss.item():.6f}, Self Acc: {self_score:.6%}, Self Loss: {self_loss:.6f}, "
            + f"Valid Acc: {last_val_score:.6%}, Valid Loss: {val_loss:.6f}", end="\n" if epoch % log_interval == 0 or epoch == num_epochs else "")

### Model Save

In [ ]:
if not os.path.isdir(os.path.join(".", "models")):
    os.mkdir(os.path.join(".", "models"))

# Model Save
save_path = os.path.join(".", "models", f"{Config.NB_NAME}_acc_{last_val_score*100:.6f}.pt")
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

## Inference
테스트 데이터셋에 대한 추론은 다음 순서로 진행됩니다.

1. 모델 및 디바이스 설정
    - 모델을 주어진 device(GPU 또는 CPU)로 이동시키고, 평가모드로 전환합니다.
2. 예측 수행
    - 예측 결과를 저장한 빈 리스트를 초기화하고 test_loader에서 배치별로 데이터를 불러와 예측을 수행합니다.
    - 각 배치에 대해 스펙트로그램 데이터를 device로 이동시킵니다.
    - 모델 예측 확률(probs)을 계산합니다.
    - 예측 확률을 predictions리스트에 추가합니다.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

In [ ]:
predicted_labels = []

model.to(device)
model.eval()
with torch.no_grad():
    for features in tqdm(test_loader):
        probs = model(features.to(device))
        probs = probs.cpu().detach().numpy()
        predicted_labels += probs.tolist()

### Submission
추론 결과를 제출 양식에 덮어 씌워 CSV 파일로 생성하는 과정은 다음과 같습니다.

1. 제출 양식 로드
    - pd.read_csv('./sample_submission.csv')를 사용하여 제출을 위한 샘플 형식 파일을 로드합니다.
    - 이 파일은 일반적으로 각 테스트 샘플에 대한 ID와 예측해야 하는 필드가 포함된 템플릿 형태를 가지고 있습니다.
2. 예측 결과 할당
    - submit.iloc[:,1:] = preds 추론함수(inference)에서 반환된 예측결과(preds)를 샘플 제출 파일에 2번째 열부터 할당합니다.
3. 제출 파일 저장
    - 수정된 제출 파일을 baseline_submit 이란 이름의 CSV 파일로 저장합니다.
    - index=False는 파일 저장시 추가적인 index가 발생하지 않도록 설정하여, 제작한 제출 파일과 동일한 형태의 파일을 저장합니다.

In [ ]:
submit = pd.read_csv(test_dataset.submission_form_path)
submit.iloc[:, 1:] = predicted_labels
submit.head()

In [ ]:
submission_dir = "submissions"
if not os.path.isdir(submission_dir):
    os.mkdir(submission_dir)

submit_file_path = os.path.join(".", submission_dir, f"{Config.NB_NAME}_acc_{last_val_score*100:.6f}_submit.csv")
submit.to_csv(submit_file_path, index=False)
print("File saved to", submit_file_path)